In [204]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

import time

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# imports best practice pandas
import os

import numpy as np
import pandas as pd

#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score

### Load sales and prices datasets, both without nulls and duplicates

In [205]:

df_sales = pd.read_csv('./data/reduced_sales.csv')


In [206]:
df_prices = pd.read_csv('./data/reduced_prices.csv')



In [207]:
del(df_sales["Unnamed: 0"])

In [208]:
df_sales

,id,date,sales
0,ACCESORIES_1_001_BOS_1,2011-01,0
1,ACCESORIES_1_001_BOS_1,2011-02,0
2,ACCESORIES_1_001_BOS_1,2011-03,0
3,ACCESORIES_1_001_BOS_1,2011-04,0
4,ACCESORIES_1_001_BOS_1,2011-05,0
...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,2015-12,41
1951356,SUPERMARKET_3_827_PHI_3,2016-01,49
1951357,SUPERMARKET_3_827_PHI_3,2016-02,46
1951358,SUPERMARKET_3_827_PHI_3,2016-03,68


In [209]:
del(df_prices["Unnamed: 0"])

In [210]:
df_prices

,item,store_code,date,sell_price
0,ACCESORIES_1_001,BOS_1,2013-07-31,11.753875
1,ACCESORIES_1_001,BOS_1,2013-08-31,10.985800
2,ACCESORIES_1_001,BOS_1,2013-09-30,10.985800
3,ACCESORIES_1_001,BOS_1,2013-10-31,10.985800
4,ACCESORIES_1_001,BOS_1,2013-11-30,10.985800
...,...,...,...,...
1536771,SUPERMARKET_3_827,PHI_3,2015-12-31,1.200000
1536772,SUPERMARKET_3_827,PHI_3,2016-01-31,1.200000
1536773,SUPERMARKET_3_827,PHI_3,2016-02-29,1.200000
1536774,SUPERMARKET_3_827,PHI_3,2016-03-31,1.200000


### Making new colunm so to merge by it

In [211]:
df_prices["id"] = df_prices["item"] + "_" + df_prices["store_code"]

In [212]:
df_prices

,item,store_code,date,sell_price,id
0,ACCESORIES_1_001,BOS_1,2013-07-31,11.753875,ACCESORIES_1_001_BOS_1
1,ACCESORIES_1_001,BOS_1,2013-08-31,10.985800,ACCESORIES_1_001_BOS_1
2,ACCESORIES_1_001,BOS_1,2013-09-30,10.985800,ACCESORIES_1_001_BOS_1
3,ACCESORIES_1_001,BOS_1,2013-10-31,10.985800,ACCESORIES_1_001_BOS_1
4,ACCESORIES_1_001,BOS_1,2013-11-30,10.985800,ACCESORIES_1_001_BOS_1
...,...,...,...,...,...
1536771,SUPERMARKET_3_827,PHI_3,2015-12-31,1.200000,SUPERMARKET_3_827_PHI_3
1536772,SUPERMARKET_3_827,PHI_3,2016-01-31,1.200000,SUPERMARKET_3_827_PHI_3
1536773,SUPERMARKET_3_827,PHI_3,2016-02-29,1.200000,SUPERMARKET_3_827_PHI_3
1536774,SUPERMARKET_3_827,PHI_3,2016-03-31,1.200000,SUPERMARKET_3_827_PHI_3


### deleting useless colunms

In [213]:
del(df_prices["item"])

In [214]:
del(df_prices["store_code"])

In [215]:
df_prices

,date,sell_price,id
0,2013-07-31,11.753875,ACCESORIES_1_001_BOS_1
1,2013-08-31,10.985800,ACCESORIES_1_001_BOS_1
2,2013-09-30,10.985800,ACCESORIES_1_001_BOS_1
3,2013-10-31,10.985800,ACCESORIES_1_001_BOS_1
4,2013-11-30,10.985800,ACCESORIES_1_001_BOS_1
...,...,...,...
1536771,2015-12-31,1.200000,SUPERMARKET_3_827_PHI_3
1536772,2016-01-31,1.200000,SUPERMARKET_3_827_PHI_3
1536773,2016-02-29,1.200000,SUPERMARKET_3_827_PHI_3
1536774,2016-03-31,1.200000,SUPERMARKET_3_827_PHI_3


### Deleting days on df["dates]

In [216]:
df_prices["date"] = df_prices["date"].astype(str).apply(lambda x: x[0:7])

In [217]:
df_prices.shape

(1536776, 3)

In [218]:
df_prices.head(5)

,date,sell_price,id
0,2013-07,11.753875,ACCESORIES_1_001_BOS_1
1,2013-08,10.985800,ACCESORIES_1_001_BOS_1
2,2013-09,10.985800,ACCESORIES_1_001_BOS_1
3,2013-10,10.985800,ACCESORIES_1_001_BOS_1
4,2013-11,10.985800,ACCESORIES_1_001_BOS_1


In [219]:
df_prices

,date,sell_price,id
0,2013-07,11.753875,ACCESORIES_1_001_BOS_1
1,2013-08,10.985800,ACCESORIES_1_001_BOS_1
2,2013-09,10.985800,ACCESORIES_1_001_BOS_1
3,2013-10,10.985800,ACCESORIES_1_001_BOS_1
4,2013-11,10.985800,ACCESORIES_1_001_BOS_1
...,...,...,...
1536771,2015-12,1.200000,SUPERMARKET_3_827_PHI_3
1536772,2016-01,1.200000,SUPERMARKET_3_827_PHI_3
1536773,2016-02,1.200000,SUPERMARKET_3_827_PHI_3
1536774,2016-03,1.200000,SUPERMARKET_3_827_PHI_3


In [220]:
df_sales

,id,date,sales
0,ACCESORIES_1_001_BOS_1,2011-01,0
1,ACCESORIES_1_001_BOS_1,2011-02,0
2,ACCESORIES_1_001_BOS_1,2011-03,0
3,ACCESORIES_1_001_BOS_1,2011-04,0
4,ACCESORIES_1_001_BOS_1,2011-05,0
...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,2015-12,41
1951356,SUPERMARKET_3_827_PHI_3,2016-01,49
1951357,SUPERMARKET_3_827_PHI_3,2016-02,46
1951358,SUPERMARKET_3_827_PHI_3,2016-03,68


In [221]:
df_full = pd.merge(df_sales,df_prices, on=["id", "date"], how="left")

### Filling nulls with 0

In [222]:
df_full["sell_price"].fillna(0, inplace=True)

###  Making new columns "Income"

In [223]:
df_full["income"] = df_full["sales"] * df_full["sell_price"]

### Loading original item_sales_df for making dicts to map new columns

In [224]:
df_full_sales = pd.read_csv("./data/item_sales.csv")

In [225]:
df_full_sales

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,2,...,2,0,0,0,0,0,1,0,0,1
30486,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,6,0,...,2,1,0,2,0,1,0,0,1,0
30488,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [226]:
store_dict = df_full_sales.groupby(["store_code"])["store"].first().to_dict()

In [227]:
region_dict = df_full_sales.groupby(["store_code"])["region"].first().to_dict()

### Making new columns from "id" column

In [228]:
df_full["store_code"] = df_full["id"].apply(lambda x: x[-5:])

In [229]:
store_dict

{'BOS_1': 'South_End',
 'BOS_2': 'Roxbury',
 'BOS_3': 'Back_Bay',
 'NYC_1': 'Greenwich_Village',
 'NYC_2': 'Harlem',
 'NYC_3': 'Tribeca',
 'NYC_4': 'Brooklyn',
 'PHI_1': 'Midtown_Village',
 'PHI_2': 'Yorktown',
 'PHI_3': 'Queen_Village'}

In [230]:
df_full["store"] = df_full["store_code"].map(store_dict)

In [231]:
df_full["region"] = df_full["store_code"].map(region_dict)

In [232]:
df_full["department"] = df_full["id"].apply(lambda x: x[:-12])

In [233]:
df_full["category"] = df_full["id"].apply(lambda x: x[:-10])

In [234]:
df_full

,id,date,sales,sell_price,income,store_code,store,region,department,category
0,ACCESORIES_1_001_BOS_1,2011-01,0,0.0,0.0,BOS_1,South_End,Boston,ACCESORIES,ACCESORIES_1
1,ACCESORIES_1_001_BOS_1,2011-02,0,0.0,0.0,BOS_1,South_End,Boston,ACCESORIES,ACCESORIES_1
2,ACCESORIES_1_001_BOS_1,2011-03,0,0.0,0.0,BOS_1,South_End,Boston,ACCESORIES,ACCESORIES_1
3,ACCESORIES_1_001_BOS_1,2011-04,0,0.0,0.0,BOS_1,South_End,Boston,ACCESORIES,ACCESORIES_1
4,ACCESORIES_1_001_BOS_1,2011-05,0,0.0,0.0,BOS_1,South_End,Boston,ACCESORIES,ACCESORIES_1
...,...,...,...,...,...,...,...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,2015-12,41,1.2,49.2,PHI_3,Queen_Village,Philadelphia,SUPERMARKET,SUPERMARKET_3
1951356,SUPERMARKET_3_827_PHI_3,2016-01,49,1.2,58.8,PHI_3,Queen_Village,Philadelphia,SUPERMARKET,SUPERMARKET_3
1951357,SUPERMARKET_3_827_PHI_3,2016-02,46,1.2,55.2,PHI_3,Queen_Village,Philadelphia,SUPERMARKET,SUPERMARKET_3
1951358,SUPERMARKET_3_827_PHI_3,2016-03,68,1.2,81.6,PHI_3,Queen_Village,Philadelphia,SUPERMARKET,SUPERMARKET_3


In [236]:
file_name = "df_full.csv"
df_full.to_csv(file_name)